# Preparação

## Bibliotecas

In [ ]:
# Extração
!pip -q install --upgrade py7zr wget

# Google CLoud: BigQuery e GCS
!pip install --upgrade -q google-cloud-bigquery google-cloud-storage

In [ ]:
# Google Colab e serviços de Cloud
from google.colab import auth
from google.cloud import storage

# Bibliotecas padrão
import pandas as pd
from pandas import DataFrame
import wget
import io
from io import StringIO
import os
from os import remove
from py7zr import SevenZipFile
import shutil

## Configurando

In [ ]:
# Autenticar
auth.authenticate_user()

In [ ]:
# Inicializa o cliente do GCS
project_id = ""
bucket_name = ''
path_raw = ''

gcs_client = storage.Client(project=project_id)
bucket = gcs_client.get_bucket(bucket_name)

# Extração

## Funções

In [ ]:
def salvar_no_gcs(arquivo_ou_df, path_blob):
    try:
        # Se for um DataFrame, salva como CSV temporário
        if isinstance(arquivo_ou_df, DataFrame):
            temp_file = "temp.csv"
            arquivo_ou_df.to_csv(temp_file, index=False)
        elif isinstance(arquivo_ou_df, str) and os.path.exists(arquivo_ou_df):
            temp_file = arquivo_ou_df
        else:
            raise ValueError("Entrada deve ser um DataFrame ou caminho de arquivo existente.")

        # Envia para o GCS
        blob = bucket.blob(path_blob)
        blob.upload_from_filename(temp_file)
        print(f"📁 Salvo em: gs://{bucket_name}/{path_blob}")

    except Exception as e:
        print("❌ Erro ao salvar no GCS:", e)

    finally:
        # Remove apenas o temporário criado internamente
        if 'temp_file' in locals() and temp_file == "temp.csv" and os.path.exists(temp_file):
            os.remove(temp_file)

In [ ]:
def extrair_e_carregar_dados_brutos(ano, mes):
    """
    Realiza o download, extração e carga dos dados brutos do Novo CAGED para o GCS.
    O período do NOVO CAGED tem início em Janeiro de 2020.
    Essa função não extrai os microdados anteriores a esse período devido a divergências metodológicas estabelecidas pelo MTE.

    São 3 tipos de arquivo: CAGEDMOV, CAGEDFOR e CAGEDEXC.
    1. CAGEDMOV: movimentações declaradas dentro do prazo com competência de declaração igual a AAAAMM.
    2. CAGEDFOR: movimentações declaradas fora do prazo com competência de declaração igual a AAAAMM.
    3. CAGEDEXC: movimentações excluídas com competência de declaração da exclusão igual a AAAAMM.

    Esta função executa o processo em memória:
    1. Baixa o arquivo .7z do FTP do MTE.
    2. Extrai o conteúdo do arquivo .txt em memória.
    3. Envia o arquivo .txt diretamente para a camada 'bruto' do GCS.

    Argumentos:
        ano (int): Ano do arquivo.
        mes (int): Mês do arquivo.
    """
    if ano < 2020:
        print(f"❌ Dados de {mes}/{ano} não pertencem ao Novo CAGED. Ignorando.")
        return

    tipos_arquivos = ['CAGEDEXC', 'CAGEDFOR', 'CAGEDMOV']
    mes_str = str(mes).zfill(2)
    pasta_temp = f"tmp_caged_{ano}{mes_str}"
    os.makedirs(pasta_temp, exist_ok=True)

    for tipo in tipos_arquivos:
        nome_base_arquivo = f'{tipo}{ano}{mes_str}'
        url = f'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO CAGED/{ano}/{ano}{mes_str}/{nome_base_arquivo}.7z'
        caminho_arquivo = os.path.join(pasta_temp, f"{nome_base_arquivo}.7z")
        caminho_extraido = os.path.join(pasta_temp, f"{nome_base_arquivo}.txt")

        print(f"--- Processando {nome_base_arquivo} ---")

        # Download do arquivo .7z em memória
        try:
            print(f'⬇️ Baixando {tipo} de {mes_str}/{ano}...')
            wget.download(url, caminho_arquivo)
        except Exception as e:
            print(f'❌ Arquivo {nome_base_arquivo} indisponível: {e}')
            continue

        # Extração em memória
        try:
            with SevenZipFile(caminho_arquivo, mode='r') as archive:
              archive.extractall(path=pasta_temp)
              print(f'✅ Extraído: {nome_base_arquivo}.txt')
        except Exception as e:
            print(f'❌ Erro na extração: {nome_base_arquivo} → {e}')
            continue

        # Upload do conteúdo extraído para o GCS
        try:
            nome_extraido = f'{tipo}{ano}{mes_str}.txt'
            path_gcs = f"{path_raw}/ano={ano}/mes={mes_str}/{nome_extraido}"
            salvar_no_gcs(caminho_extraido, path_gcs)
        except Exception as e:
            print(f'❌ Erro ao enviar para GCS: {e}')
            continue

    # Remoção do arquivo .7z e .txt após upload
    try:
        shutil.rmtree(pasta_temp)
        print(f"🗑️ Pasta temporária removida: {pasta_temp}")
    except Exception as e:
        print(f"⚠️ Erro ao remover pasta temporária: {e}")

## Extração Completa (2020 - Hoje)

In [ ]:
# Extraindo arquivos para todo o período
for ano in range(2020, 2026):
    for mes in range(1, 13):
        extrair_e_carregar_dados_brutos(ano, mes)

## Extração Específica

In [ ]:
# Extraindo arquivos de um mês específico
extrair_e_carregar_dados_brutos(2025, 5)